### Analysis of Deaf Schools in America, 2026

In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
import seaborn as sns
import folium as fm
import matplotlib.pyplot as plt
from shapely.geometry import Point

##### Load data

In [2]:
schools = pd.read_csv('../data/deaf_schools/clean/deaf_schools2026.csv',index_col=False)

##### Basic EDA

In [3]:
schools.head()

,name,yr_est,yrs_opr,address,city,state,zip,mascot,gr_served,day_or_res,website,lat,long
0,Alabama Institute for the Deaf and Blind,1858,168,205 South Street East,Talladega,Alabama,35160,Silent Warriors,PreK-12,Residential,https://www.aidb.org,33.43083,-86.0992
1,Alaska State School for the Deaf and Hard of H...,1973,53,5530 E Northern Lights Blvd,Anchorage,Alaska,99504,Seals,PreK-12,Residential,https://www.asdk12.org/aksdhh,61.19399,-149.7750
2,American School for the Deaf,1817,209,139 North Main Street,West Hartford,Connecticut,06107,Tigers,K-12,Residential,http://www.asd-1817.org/,41.77111,-72.7471
3,Arizona State Schools for the Deaf and Blind,1912,114,1200 W Speedway Blvd,Tucson,Arizona,85745,Sentinels,PreK-12,Residential,https://asdb.az.gov/tucsoncampus/,32.23709,-110.9900
4,Arkansas School for the Deaf,1849,177,2400 W Markham St,Little Rock,Arkansas,72205,Leopards,PreK-12,Residential,https://asd.ade.arkansas.gov/,34.75164,-92.2978


In [4]:
schools.shape

(71, 13)

In [5]:
schools.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   name        71 non-null     object 
 1   yr_est      71 non-null     int64  
 2   yrs_opr     71 non-null     int64  
 3   address     71 non-null     object 
 4   city        71 non-null     object 
 5   state       71 non-null     object 
 6   zip         71 non-null     object 
 7   mascot      65 non-null     object 
 8   gr_served   71 non-null     object 
 9   day_or_res  71 non-null     object 
 10  website     71 non-null     object 
 11  lat         71 non-null     float64
 12  long        71 non-null     float64
dtypes: float64(2), int64(2), object(9)
memory usage: 7.3+ KB


In [6]:
schools.describe()

,yr_est,yrs_opr,lat,long
count,71.000000,71.000000,71.000000,71.000000
mean,1895.535211,130.464789,39.250226,-90.439317
std,53.967935,53.967935,5.239260,18.625034
min,1817.000000,6.000000,21.273350,-157.815000
25%,1854.500000,86.500000,35.728630,-98.308050
50%,1876.000000,150.000000,39.834780,-84.444600
75%,1939.500000,171.500000,41.940940,-76.250500
max,2020.000000,209.000000,61.193990,-70.232700


In [7]:
schools.columns.tolist()

['name',
 'yr_est',
 'yrs_opr',
 'address',
 'city',
 'state',
 'zip',
 'mascot',
 'gr_served',
 'day_or_res',
 'website',
 'lat',
 'long']

In [8]:
schools.nunique()

name          71
yr_est        58
yrs_opr       58
address       69
city          67
state         45
zip           69
mascot        45
gr_served      6
day_or_res     2
website       69
lat           71
long          71
dtype: int64

##### Geospatial Analysis

In [9]:
# lat/long to Points
geometry = [Point(xy) for xy in zip(schools['long'], schools['lat'])]

In [10]:
# geopandas dataframe
gdf = gpd.GeoDataFrame(schools, geometry=geometry)

In [11]:
# set coordinate ref system to WGS84
gdf.set_crs("EPSG:4326", allow_override=True, inplace=True)

,name,yr_est,yrs_opr,address,city,state,zip,mascot,gr_served,day_or_res,website,lat,long,geometry
0,Alabama Institute for the Deaf and Blind,1858,168,205 South Street East,Talladega,Alabama,35160,Silent Warriors,PreK-12,Residential,https://www.aidb.org,33.43083,-86.0992,POINT (-86.0992 33.43083)
1,Alaska State School for the Deaf and Hard of H...,1973,53,5530 E Northern Lights Blvd,Anchorage,Alaska,99504,Seals,PreK-12,Residential,https://www.asdk12.org/aksdhh,61.19399,-149.7750,POINT (-149.775 61.19399)
2,American School for the Deaf,1817,209,139 North Main Street,West Hartford,Connecticut,06107,Tigers,K-12,Residential,http://www.asd-1817.org/,41.77111,-72.7471,POINT (-72.7471 41.77111)
3,Arizona State Schools for the Deaf and Blind,1912,114,1200 W Speedway Blvd,Tucson,Arizona,85745,Sentinels,PreK-12,Residential,https://asdb.az.gov/tucsoncampus/,32.23709,-110.9900,POINT (-110.99 32.23709)
4,Arkansas School for the Deaf,1849,177,2400 W Markham St,Little Rock,Arkansas,72205,Leopards,PreK-12,Residential,https://asd.ade.arkansas.gov/,34.75164,-92.2978,POINT (-92.2978 34.75164)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,West Virginia Schools for the Deaf and Blind,1870,156,301 East Main Street,Romney,West Virginia,26757,Lions,PreK-12,Residential,https://www.wvsdb2.state.k12.wv.us/,39.34160,-78.7520,POINT (-78.752 39.3416)
67,Western Pennsylvania School for the Deaf,1869,157,300 East Swissvale Avenue,Edgewood,Pennsylvania,15218,Lions,PreK-12,Residential,https://www.wpsd.org/,40.43457,-79.8815,POINT (-79.8815 40.43457)
68,West Tennessee School for the Deaf,1986,40,100 Berryhill Drive,Jackson,Tennessee,38301,Eagles,PreK-6,Day,https://www.tsdeaf.org/apps/pages/WTSD,35.64632,-88.7887,POINT (-88.7887 35.64632)
69,Willie Ross School for the Deaf,1967,59,32 Norway Street,Longmeadow,Massachusetts,01106,NaN,PreK-12,Day,https://www.wrsdeaf.org/,42.03104,-72.5782,POINT (-72.5782 42.03104)


In [21]:
m = fm.Map(
    location=[37.0902, -95.7129],
    zoom_start=3,
    tiles='OpenStreetMap'
)

gdf.explore(
    m=m,
    color="red",
    marker_kwds=dict(radius=5, fill=True),
    tooltip=['name'],
    popup=['name', 'address', 'city', 'state', 'zip', 'yrs_opr', 'mascot', 'website'], 
    tooltip_kwds=dict(labels=False),
    name="deaf_schools",
)

m

##### Visualizations

In [20]:
schools.columns.tolist()

['name',
 'yr_est',
 'yrs_opr',
 'address',
 'city',
 'state',
 'zip',
 'mascot',
 'gr_served',
 'day_or_res',
 'website',
 'lat',
 'long']